# Quantile QTL Association Testing

## Description

We perform QTL association testing using quantile regression according to [this paper](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7673343/).

## Input

- List of molecular phenotype files: a list of `bed.gz` files containing the table for the molecular phenotype. It should have a companion index file in `tbi` format. It is the output of gene_annotation or phenotype_by_chorm

### Example phenotype list

```
#chr    start   end ID  path
chr12   752578  752579  ENSG00000060237  /home/gw/GIT/github/fungen-xqtl-analysis/analysis/Wang_Columbia/ROSMAP/MWE/output/phenotype/protocol_example.protein.bed.gz
chr12   990508  990509  ENSG00000082805  /home/gw/GIT/github/fungen-xqtl-analysis/analysis/Wang_Columbia/ROSMAP/MWE/output/phenotype/protocol_example.protein.bed.gz
chr12   2794969 2794970 ENSG00000004478  /home/gw/GIT/github/fungen-xqtl-analysis/analysis/Wang_Columbia/ROSMAP/MWE/output/phenotype/protocol_example.protein.bed.gz
chr12   4649113 4649114 ENSG00000139180  /home/gw/GIT/github/fungen-xqtl-analysis/analysis/Wang_Columbia/ROSMAP/MWE/output/phenotype/protocol_example.protein.bed.gz
chr12   6124769 6124770 ENSG00000110799  /home/gw/GIT/github/fungen-xqtl-analysis/analysis/Wang_Columbia/ROSMAP/MWE/output/phenotype/protocol_example.protein.bed.gz
chr12   6534516 6534517 ENSG00000111640  /home/gw/GIT/github/fungen-xqtl-analysis/analysis/Wang_Columbia/ROSMAP/MWE/output/phenotype/protocol_example.protein.bed.gz
```


    The header of the bed.gz is per the [TensorQTL](https://github.com/broadinstitute/tensorqtl) convention:

    >    Phenotypes must be provided in BED format, with a single header line starting with # and the first four columns corresponding to: chr, start, end, phenotype_id, with the remaining columns corresponding to samples (the identifiers must match those in the genotype input). The BED file should specify the center of the cis-window (usually the TSS), with start == end-1.


- List of genotypes in PLINK binary format (`bed`/`bim`/`fam`) for each chromosome, previously processed through our genotype QC pipelines.
- Covariate file, a file with #id + samples name as colnames and each row a covariate: fixed and known covariates as well as hidden covariates recovered from factor analysis.
- Optionally, a list of traits (genes, regions of molecular features etc) to analyze.


For cis-analysis:

- Optionally, a list of genomic regions associate with each molecular features to analyze. The default cis-analysis will use a window around TSS. This can be customized to take given start and end genomic coordinates.

## Output

For each chromosome, several of summary statistics files are generated, including both nominal test statistics for each test, as well as region (gene) level association evidence.

The columns of nominal association result are as follows:
- phenotype_id: Molecular trait identifier.(gene)
- variant_id_id: ID of the variant (rsid or chr:position:ref:alt)
- chr : Variant chromosome.
- pos : Variant chromosomal position (basepairs).
- ref : Variant reference allele (A, C, T, or G).
- alt : Variant alternate allele.
- combined_pval(composite-p value using cauchy combination method): the integrated QR p-value across multiple quantile levels.    
- qr_0.1_pval to qr_0.9_pval: quantile-specific QR p-values for the quantile levels 0.1, 0.2, ..., 0.9.   
- qr_0.1_slope to qr_0.9_slope: quantile-specific QR coefficients for the quantile levels 0.1, 0.2, ..., 0.9.  
- qr_0.1_tval to qr_0.9_tval: quantile-specific QR t-values for the quantile levels 0.1, 0.2, ..., 0.9.

## Minimal Working Example Steps



The data can be found on [Synapse](https://www.synapse.org/#!Synapse:syn36416559/files/).

### i. Cis TensorQTL Command 

In [ ]:
sos run pipeline/quantile_qtl_python.ipynb quant \
    --genotype-file output/genotype_by_chrom/protocol_example.genotype.chr21_22.genotype_by_chrom_files.txt \
    --phenotype-file  output/phenotype_by_chrom/protocol_example.protein.bed.phenotype_by_chrom_files.region_list.txt \
    --covariate-file output/covariate/protocol_example.protein.protocol_example.samples.protocol_example.genotype.chr21_22.pQTL.plink_qc.prune.pca.Marchenko_PC.gz \
    --customized_cis_windows prototype_example/protocol_example/protocol_example.protein.enhanced_cis_chr21_chr22.bed \
    --cwd output/cis_association/ \
    --MAC 5 \
    --container containers/TensorQTL.sif 

In [ ]:
zcat output/protocol_example.protein.bed.gz | cut -f 1,2,3,4 | grep -v -e ENSG00000163554 \
    -e ENSG00000171564 -e ENSG00000171560 -e ENSG00000171557 > output/protocol_example.protein.region_list

## Troubleshooting

| Step | Substep | Problem | Possible Reason | Solution |
|------|---------|---------|------------------|---------|
|  |  |  |  |  |




## Setup and global parameters

In [3]:
[global]
# Path to the work directory of the analysis.
parameter: cwd = path('output')
# Phenotype file, or a list of phenotype per region.
parameter: phenotype_file = path
# A genotype file in PLINK binary format (bed/bam/fam) format, or a list of genotype per chrom
parameter: genotype_file = path
# Covariate file
parameter: covariate_file = path
# Prefix for the analysis output
parameter: name = f"{phenotype_file:bn}_{covariate_file:bn}"
# An optional subset of regions of molecular features to analyze. The last column is the gene names
parameter: region_list = path()
parameter: region_list_phenotype_column = 4
# Set list of sample to be keep
parameter: keep_sample = path()
# FIXME: please document
parameter: interaction = ""

# An optional list documenting the custom cis window for each region to analyze, with four column, chr, start, end, region ID (eg gene ID).
# If this list is not provided, the default `window` parameter (see below) will be used.
parameter: customized_cis_windows = path()

# The phenotype group file to group molecule_trait into molecule_trait_object
# This applies to multiple molecular events in the same region, such as sQTL analysis.
parameter: phenotype_group = path() 

# The name of phenotype corresponding to gene_id or gene_name in the region
parameter: chromosome = []
# Minor allele count cutoff
parameter: MAC = 0

# Specify the cis window for the up and downstream radius to analyze around the region of interest in units of bp
# This parameter will be set to zero if `customized_cis_windows` is provided.
parameter: window = 1000000

# Number of threads
parameter: numThreads = 8
# For cluster jobs, number commands to run per job
parameter: job_size = 1
parameter: walltime = '12h'
parameter: mem = '16G'
# Container option for software to run the analysis: docker or singularity
parameter: container = ''
import re
parameter: entrypoint= ('micromamba run -a "" -n' + ' ' + re.sub(r'(_apptainer:latest|_docker:latest|\.sif)$', '', container.split('/')[-1])) if container else ""

# Use the header of the covariate file to decide the sample size
import pandas as pd
N = len(pd.read_csv(covariate_file, sep = "\t",nrows = 1).columns) - 1

# Minor allele frequency cutoff. It will overwrite minor allele cutoff.
# You may consider setting it to higher for interaction analysis if you have statistical power concerns
parameter: maf_threshold = MAC/(2.0*N) 

import os
import pandas as pd

def adapt_file_path(file_path, reference_file):
    """
    Adapt a single file path based on its existence and a reference file's path.

    Args:
    - file_path (str): The file path to adapt.
    - reference_file (str): File path to use as a reference for adaptation.

    Returns:
    - str: Adapted file path.

    Raises:
    - FileNotFoundError: If no valid file path is found.
    """
    reference_path = os.path.dirname(reference_file)

    # Check if the file exists
    if os.path.isfile(file_path):
        return file_path

    # Check file name without path
    file_name = os.path.basename(file_path)
    if os.path.isfile(file_name):
        return file_name

    # Check file name in reference file's directory
    file_in_ref_dir = os.path.join(reference_path, file_name)
    if os.path.isfile(file_in_ref_dir):
        return file_in_ref_dir

    # Check original file path prefixed with reference file's directory
    file_prefixed = os.path.join(reference_path, file_path)
    if os.path.isfile(file_prefixed):
        return file_prefixed

    # If all checks fail, raise an error
    raise FileNotFoundError(f"No valid path found for file: {file_path}")

def adapt_file_path_all(df, column_name, reference_file):
    return df[column_name].apply(lambda x: adapt_file_path(x, reference_file))


if str(genotype_file).endswith("bed") and str(phenotype_file).endswith("bed.gz"):
    input_files = [[phenotype_file, genotype_file]]
    input_chroms = [0]
else:
    import pandas as pd
    import os
    molecular_pheno_files = pd.read_csv(phenotype_file, sep = "\t")
    if "#chr" in molecular_pheno_files.columns:
        molecular_pheno_files = molecular_pheno_files.groupby(['#chr', '#dir']).size().reset_index(name='count').drop("count",axis = 1).rename(columns = {"#chr":"#id"})
    genotype_files = pd.read_csv(genotype_file,sep = "\t")
    genotype_files["#id"] = [x.replace("chr","") for x in genotype_files["#id"].astype(str)] # e.g. remove chr1 to 1
    genotype_files["#path"] = genotype_files["#path"].apply(lambda x: adapt_file_path(x, genotype_file))
    molecular_pheno_files["#id"] = [x.replace("chr","") for x in molecular_pheno_files["#id"].astype(str)]
    input_files = molecular_pheno_files.merge(genotype_files, on = "#id")
    # Only keep chromosome specified in --chromosome
    if len(chromosome) > 0:
        input_files = input_files[input_files['#id'].isin(chromosome)]
    input_files = input_files.values.tolist()
    input_chroms = [x[0] for x in input_files]
    input_files = [x[1:] for x in input_files]
    print("Input Files:", input_files)
    print("Input Chromosomes:", input_chroms)

## Quantile qtl cis association testing: by chromosome

In [ ]:
# e.g. mega data:exc
nohup sos run pipeline/quantile_qtl_python.ipynb quant \
    --name snuc_pseudo_bulk_mega_exc \
    --genotype-file /mnt/vast/hpc/csg/FunGen_xQTL/ROSMAP/Genotype/geno_by_chrom/ROSMAP_NIA_WGS.leftnorm.bcftools_qc.plink_qc.genotype_by_chrom_files.txt \
    --phenotype-file /mnt/vast/hpc/csg/wanggroup/fungen-xqtl-analysis/analysis/Wang_Columbia/ROSMAP/pseudo_bulk_eqtl_mega/Exc/output/data_preprocessing/phenotype_data/phenotype_by_chrom/snuc_pseudo_bulk.Exc.mega.normalized.log2cpm.bed.phenotype_by_chrom_files.txt \
    --region-list /home/al4225/project/quantile_qtl/python_version/combined_AD_genes.csv \
    --covariate-file /mnt/vast/hpc/csg/wanggroup/fungen-xqtl-analysis/analysis/Wang_Columbia/ROSMAP/pseudo_bulk_eqtl_mega/Exc/output/data_preprocessing/covariate_data/snuc_pseudo_bulk.Exc.mega.normalized.log2cpm.rosmap_cov.ROSMAP_NIA_WGS.leftnorm.bcftools_qc.plink_qc.snuc_pseudo_bulk_mega.related.plink_qc.extracted.pca.projected.Marchenko_PC.gz \
    --customized_cis_windows /home/al4225/project/fungen-xqtl-analysis/resource/TADB_enhanced_cis.bed \
    --cwd /home/al4225/project/quantile_qtl/pseudobulk_mega/output/exc/ \
    --container /mnt/vast/hpc/csg/containers_xqtl/TensorQTL.sif --maf-threshold 0.05 \
    --mem 60G -J 50 -c /mnt/vast/hpc/csg/molecular_phenotype_calling/csg.yml -q csg

In [ ]:
[quant]
# parse input file lists
# skip nominal association results if the files exists already
# This is false by default which means to recompute everything
# This is only relevant when the `parquet` files for nominal results exist but not the other files
# and you want to avoid computing the nominal results again
#parameter: skip_nominal_if_exist = False
#parameter: permutation = True
parameter: vcov= 'robust'
parameter: kernel='epa'
parameter: bandwidth='hsheather'
parameter: max_iter=1000
parameter: p_tol=0.000001

# Extract interaction name
var_interaction = interaction
if os.path.isfile(interaction):
    interaction_s = pd.read_csv(interaction, sep='\t', index_col=0)
    var_interaction = interaction_s.columns[0] # interaction name
test_regional_association = permutation and len(var_interaction) == 0

input: input_files, group_by = len(input_files[0]), group_with = "input_chroms"
output_files = dict([("parquet", f'{cwd:a}/{_input[0]:bnn}{"_%s" % var_interaction if interaction else ""}.cis_qtl_pairs.{"" if input_chroms[_index] == 0 else input_chroms[_index]}.parquet'), # This convention is necessary to match the pattern of map_norminal output
                     ("nominal", f'{cwd:a}/{_input[0]:bnn}{"" if input_chroms[_index] == 0 else "_chr%s" % input_chroms[_index]}{"_%s" % var_interaction if interaction else ""}.cis_qtl.pairs.tsv.gz')])
if test_regional_association:
    output_files["regional"] = f'{cwd:a}/{_input[0]:bnn}{"" if input_chroms[_index] == 0 else "_chr%s" % input_chroms[_index]}{"_%s" % var_interaction if interaction else ""}.cis_qtl.regional.tsv.gz'
output: output_files, quantileqtl_nominal_results = f'{cwd:a}/{name}.{_input_chroms}.quantileqtl_nominal_results.txt'
task: trunk_workers = 1, trunk_size = job_size, walltime = walltime, mem = mem, cores = numThreads, tags = f'{step_name}_{_output["nominal"]:bnnn}'
python: expand= "$[ ]", stderr = f'{_output["nominal"]:nnn}.stderr', stdout = f'{_output["nominal"]:nnn}.stdout' , container = container, entrypoint = entrypoint
    import pandas as pd
    import numpy as np
    import os
    import tensorqtl
    from tensorqtl import genotypeio, cis
    from scipy.stats import chi2
    import statsmodels.api as sm
    import scipy.stats as ss
    from scipy.stats import cauchy
    import time

    start_time = time.time()

    def calculate_maf(genotype_array, alleles=2):
        af_array = genotype_array.sum(axis=1) / (alleles * genotype_array.shape[1])
        maf_array = np.where(af_array > 0.5, 1 - af_array, af_array)
        return maf_array

    def cauchy_meta(pvals):
        # Check input
        pvals = np.array(pvals)
        pvals = pvals[~np.isnan(pvals)]
        if len(pvals) == 0:
            return np.nan
        # pvals[pvals == 0] = 2.2e-308
        # Convert to Cauchy
        cauchy_vals = np.zeros(pvals.shape)
        valid_indices = pvals >= 1e-15
        cauchy_vals[valid_indices] = np.tan(np.pi * (0.5 - pvals[valid_indices]))
        stats = np.mean(cauchy_vals)
        p = cauchy.sf(stats)
        return p

    def fit_quantreg_with_covariates(phenotype_id, phenotype_array, genotype_matrix, covar_df, variant_ids, taus=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9], vcov='$[vcov]', kernel='$[kernel]', bandwidth='$[bandwidth]', max_iter=$[max_iter], p_tol=$[p_tol]):
        results = []
        variant_pvals = {vid: [] for vid in variant_ids}  # Prepare a dict to store p-values for combined p-value calculation

        # Ensure the correct orientation of genotype_matrix to match the samples in covar_df
        if genotype_matrix.shape[1] != covar_df.shape[0]: #covar: row: SM, geno: col:SM
            raise ValueError("Mismatch in the number of samples between genotype matrix and covariates DataFrame")

        for tau in taus:
            for idx, genotype_row in enumerate(genotype_matrix): #genotype_matrix is a 2-dimension numpy,iterate over variants
                X = np.c_[covar_df.values, genotype_row] 
                X = sm.add_constant(X).astype('float')
                model = sm.QuantReg(phenotype_array, X)
                try:
                    res = model.fit(q=tau, vcov=vcov, kernel=kernel, bandwidth=bandwidth, max_iter=max_iter, p_tol=p_tol)
                    pval = res.pvalues[-1]
                    variant_pvals[variant_ids[idx]].append(pval)
                    results.append({
                        'phenotype_id': phenotype_id,
                        'variant_id': variant_ids[idx],
                        'tau': tau,
                        'pval': pval,
                        'slope': res.params[-1],
                        'tval': res.tvalues[-1]
                    })
                except Exception as e:
                    print(f"Error fitting model for variant {variant_ids[idx]} at tau {tau}: {e}")
                if (idx + 1) % 100 == 0:
                    print(f"{idx+1} pairs, percentage {idx+1}/{len(variant_ids)} for phenotype_id {phenotype_id} finished.")
        # Calculate combined p-values for each variant
        for vid in variant_ids:
            combined_pval = cauchy_meta(variant_pvals[vid])
            for result in filter(lambda r: r['variant_id'] == vid, results):
                result['combined_pval'] = combined_pval
        results_df = pd.DataFrame(results)
        
        wide_df = pd.pivot_table(results_df, values=['pval', 'slope', 'tval'], index=['phenotype_id', 'variant_id', 'combined_pval'], columns='tau')
        wide_df.columns = ['qr_{}_{}'.format(col[1], col[0]) for col in wide_df.columns]
        wide_df.reset_index(inplace=True)
        variant_split = wide_df['variant_id'].str.extract(r'chr(\d+):(\d+)_([A-Z*]+)_([A-Z*]+)')
        variant_split.columns = ['chr', 'pos', 'ref', 'alt']
        final_df = pd.concat([variant_split, wide_df], axis=1)
        
        return final_df

        
    ## Define paths
    plink_prefix_path = $[_input[1]:nar]
    expression_bed = $[_input[0]:ar]
    covariates_file = "$[covariate_file:a]"
    window = $[window]
    interaction = "$[interaction]"
    ## Load Data
    phenotype_df, phenotype_pos_df = tensorqtl.read_phenotype_bed(expression_bed)

    phenotype_id = phenotype_pos_df.index.name
    ## Analyze only the regions listed
    if $[region_list.is_file()]:
        region = pd.read_csv("$[region_list:a]", comment="#", header=None, sep="," )
        phenotype_column = 1 if len(region.columns) == 1 else  $[region_list_phenotype_column]
        keep_region = region.iloc[:,phenotype_column-1].to_list()
        phenotype_df = phenotype_df[phenotype_df.index.isin(keep_region)]
        phenotype_pos_df = phenotype_pos_df[phenotype_pos_df.index.isin(keep_region)]

    covariates_df = pd.read_csv(covariates_file, sep='\t', index_col=0).T

    pr = genotypeio.PlinkReader(plink_prefix_path)
    genotype_df = pr.load_genotypes()

    # keep 1st 2k cols
    genotype_df = genotype_df.iloc[:10000, :]

    variant_df = pr.bim.set_index('snp')[['chrom', 'pos','a0','a1']]
    variant_df = variant_df.iloc[:10000, :]
    
    ## use custom sample list to subset the covariates data
    if $[keep_sample.is_file()]:
        sample_list = pd.read_csv("$[keep_sample:a]", comment="#", header=None, names=["sample_id"], sep="\t")
        covariates_df.loc[sample_list.sample_id]

    # Read interaction files or extract from covariates file.
    var_interaction = interaction
    interaction_s = []
    if os.path.isfile(interaction):
        # update var_interaction and interaction_s
        interaction_s = pd.read_csv(interaction, sep='\t', index_col=0)
        interaction_s = interaction_s[interaction_s.index.isin(covariates_df.index)] 
        var_interaction = interaction_s.columns[0] # interaction name
    # check if the interaction term in interaction table is in covariates file, if yes and interaction_s not yet loaded then, extract it out from covariates file
    if var_interaction in covariates_df.columns:
        # only load from covariate if it has not been loaded yet
        if len(interaction_s) == 0:
            interaction_s = covariates_df[var_interaction].to_frame()
        covariates_df = covariates_df.drop(columns=[var_interaction])
    if len(interaction) and len(interaction_s) == 0:
        raise ValueError(f"Cannot find interaction variable or file {interaction}")

    # drop samples that with missing value in iteraction
    if len(interaction_s):
        interaction_s = interaction_s.dropna() 

    ## Retaining only common samples
    phenotype_df.columns = phenotype_df.columns.astype(str)
    covariates_df.index = covariates_df.index.astype(str)

    phenotype_df = phenotype_df[np.intersect1d(phenotype_df.columns, covariates_df.index)]
    phenotype_df = phenotype_df[np.intersect1d(phenotype_df.columns, genotype_df.columns)]
    if len(interaction_s):
        phenotype_df = phenotype_df[np.intersect1d(phenotype_df.columns, interaction_s.index)]
        interaction_s = interaction_s[interaction_s.index.isin(phenotype_df.columns)]    
        interaction_s = interaction_s.loc[phenotype_df.columns]

    covariates_df = covariates_df.transpose()[np.intersect1d(phenotype_df.columns, covariates_df.index)].transpose()
    ## To simplify things, there should really not be "chr" prefix
    phenotype_pos_df.chr = phenotype_pos_df.chr.astype(str).str.replace("chr", "")
    variant_df.chrom =  variant_df.chrom.astype("str").str.replace("chr", "") 
    
    common_samples = np.intersect1d(phenotype_df.columns, genotype_df.columns)
    genotype_df = genotype_df[common_samples]
    # same sample order
    sample_order = covariates_df.index
    phenotype_df = phenotype_df[sample_order]
    genotype_df = genotype_df[sample_order]

    ## use custom cis windows list
    if $[customized_cis_windows.is_file()]:
        cis_list = pd.read_csv("$[customized_cis_windows:a]", comment="#", header=None, names=["chr","start","end",phenotype_id], sep="\t")
        cis_list.chr = cis_list.chr.astype(str).str.replace("chr", "")  ## Again to simplify things for chr format concordance.
        phenotype_pos_df = phenotype_pos_df.reset_index() #move the phenotype id index to a new column of the dataframe
        phenotype_pos_df = phenotype_pos_df.merge(cis_list, left_on = ["chr",phenotype_id],right_on = [cis_list.columns[0],cis_list.columns[3]])#in some cases (gene expression for eQTLs) the phenotype_id may be in the cis_list file
        phenotype_pos_df = phenotype_pos_df.set_index(phenotype_id)[["chr","start","end"]] # The final phenotype_pos_df will have three columns(chr, start, end) and index is the phenotype ID
        if len(phenotype_df.index) != len(phenotype_pos_df.index):
            raise ValueError("cannot uniquely match all the phentoype data in the input to the customized cis windows provided")
        window = 0 
 
    ## Read phenotype group if availble
    if $[phenotype_group.is_file()]:
        group_s = pd.read_csv($[phenotype_group:r], sep='\t', header=None, index_col=0, squeeze=True)
    else:
        group_s = None

    # quantile qtl mapping:
    igc = genotypeio.InputGeneratorCis(genotype_df, variant_df, phenotype_df, phenotype_pos_df, group_s=group_s, window=window)

    results_list = [] 

    for phenotype, genotypes, genotype_range, phenotype_id in igc.generate_data(chrom=chrom, verbose=True):
        variant_ids = variant_df.index[genotype_range[0]:genotype_range[-1]+1].tolist()
        if $[maf_threshold] > 0:
            maf = calculate_maf(genotypes)
            mask = maf >= $[maf_threshold]
            genotypes = genotypes[mask]
            mask = mask.astype(bool)
            variant_ids = np.array(variant_ids)[mask]
            print("next step is : quantreg")

        results_df = fit_quantreg_with_covariates(phenotype_id, phenotype, genotypes, covariates_df, variant_ids)
        results_list.append(results_df)

    all_results_df = pd.concat(results_list, ignore_index=True)
    all_results_df.to_csv("$[_output['quantileqtl_nominal_results']]", sep="\t", index=False)
    print(f"quantlie qtl completed")
    end_time = time.time()  
    print(f"Total execution time: {end_time - start_time} seconds")